# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [36]:
# import libraries
import numpy as np
import pandas as pd
import nltk
nltk.download(['punkt','wordnet'])
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sjhal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sjhal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [37]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('DisasterResponse',engine)
X = df[['message']].message
Y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [38]:
stop_words = stopwords.words("english")

def tokenize(text):
    words = word_tokenize(text.lower())
    lemmatizer = WordNetLemmatizer()
    
    tokens = [lemmatizer.lemmatize(word).lower().strip() for word in words if word not in stop_words]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [39]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [40]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)
pipeline.fit(X_train,Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001F7A8F769D0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [41]:
y_pred = pipeline.predict(X_test)

In [42]:
y_pred = pd.DataFrame(y_pred,columns=Y_test.columns)

In [43]:
for col in Y_test.columns:
    print(col)
    print(classification_report(Y_test[col], y_pred[col]))

related
              precision    recall  f1-score   support

         0.0       0.40      0.04      0.08      1503
         1.0       0.77      0.98      0.86      5000
         2.0       0.00      0.00      0.00        49

    accuracy                           0.76      6552
   macro avg       0.39      0.34      0.31      6552
weighted avg       0.68      0.76      0.67      6552

request
              precision    recall  f1-score   support

         0.0       0.83      0.99      0.90      5395
         1.0       0.37      0.03      0.06      1157

    accuracy                           0.82      6552
   macro avg       0.60      0.51      0.48      6552
weighted avg       0.75      0.82      0.75      6552

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6530
         1.0       0.00      0.00      0.00        22

    accuracy                           1.00      6552
   macro avg       0.50      0.50      0.50      655

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.88      1.00      0.94      5790
         1.0       0.05      0.00      0.00       762

    accuracy                           0.88      6552
   macro avg       0.47      0.50      0.47      6552
weighted avg       0.79      0.88      0.83      6552

shelter
              precision    recall  f1-score   support

         0.0       0.91      1.00      0.95      5953
         1.0       0.04      0.00      0.00       599

    accuracy                           0.90      6552
   macro avg       0.47      0.50      0.48      6552
weighted avg       0.83      0.90      0.86      6552

clothing
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6457
         1.0       0.00      0.00      0.00        95

    accuracy                           0.99      6552
   macro avg       0.49      0.50      0.50      6552
weighted avg       0.97      0.99      0.98      6552

mo

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.95      1.00      0.98      6255
         1.0       0.33      0.01      0.01       297

    accuracy                           0.95      6552
   macro avg       0.64      0.50      0.49      6552
weighted avg       0.93      0.95      0.93      6552

other_aid
              precision    recall  f1-score   support

         0.0       0.87      0.99      0.93      5710
         1.0       0.08      0.01      0.01       842

    accuracy                           0.86      6552
   macro avg       0.48      0.50      0.47      6552
weighted avg       0.77      0.86      0.81      6552

infrastructure_related
              precision    recall  f1-score   support

         0.0       0.94      1.00      0.97      6127
         1.0       0.11      0.00      0.00       425

    accuracy                           0.93      6552
   macro avg       0.52      0.50      0.49      6552
weighted avg       0.88      0.93      0.

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjha

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6523
         1.0       0.00      0.00      0.00        29

    accuracy                           1.00      6552
   macro avg       0.50      0.50      0.50      6552
weighted avg       0.99      1.00      0.99      6552

hospitals
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      6489
         1.0       0.00      0.00      0.00        63

    accuracy                           0.99      6552
   macro avg       0.50      0.50      0.50      6552
weighted avg       0.98      0.99      0.99      6552

shops
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6523
         1.0       0.00      0.00      0.00        29

    accuracy                           1.00      6552
   macro avg       0.50      0.50      0.50      6552
weighted avg       0.99      1.00      0.99      6552

aid

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjha

In [45]:
pipeline.score(X_test, Y_test)

0.15842490842490842

In [73]:
np.mean(np.mean(Y_test.reset_index(drop=True).eq(y_pred), axis=0))

0.9249974562474562

In [102]:
df_add = Y_test.reset_index(drop=True).add(y_pred, fill_value=0)
(df_add[df_add == 2]/2).sum(axis=0, numeric_only=True).sum()/(Y_test.sum(axis=0, numeric_only=True).sum())

0.27947202897169543

### 6. Improve your model
Use grid search to find better parameters. 

In [46]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001F7A8F769D0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001F7A8F769D0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [74]:
'''
First Run of GridSearchCV on RandomForestClassifier
1/18/21 11AM
parameters = {'vect__max_df': [0.9, 1.0],
              'vect__min_df': [0.0, 0.1],
              'vect__max_features': [500, 1000, None],
              'clf__estimator__n_estimators': [50, 100, 250]}
              
The best parameters across ALL searched params:
{'clf__estimator__n_estimators': 250, 'vect__max_df': 0.9, 'vect__max_features': None, 'vect__min_df': 0.0}
'''

'''
Second Run of GridSearchCV on RandomForestClassifier
1/18/21 4PM
parameters = {'vect__max_df': [0.8, 0.9],
              'clf__estimator__n_estimators': [250, 400]}
              
 The best parameters across ALL searched params:
 {'clf__estimator__n_estimators': 400, 'vect__max_df': 0.8}
'''

parameters = {'vect__max_df': [0.7, 0.8],
              'clf__estimator__n_estimators': [400]}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, n_jobs=6, cv=3, verbose=5)

In [75]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001F7A8F769D0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             n_jobs=6,
             param_grid={'clf__estimator__n_estimators': [400],
                         'vect__max_df': [0.7, 0.8]},
             verbose=5)

In [76]:
print(" Results from Grid Search (model name: cv)" )
print("\n The best estimator across ALL searched params:\n",cv.best_estimator_)
print("\n The best score across ALL searched params:\n",cv.best_score_)
print("\n The best parameters across ALL searched params:\n",cv.best_params_)

 Results from Grid Search (model name: cv)

 The best estimator across ALL searched params:
 Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.8,
                                 tokenizer=<function tokenize at 0x000001F7A8F769D0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_estimators=400)))])

 The best score across ALL searched params:
 0.16413120820844831

 The best parameters across ALL searched params:
 {'clf__estimator__n_estimators': 400, 'vect__max_df': 0.8}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [77]:
y_pred_cv = cv.predict(X_test)

In [78]:
y_pred_cv = pd.DataFrame(y_pred_cv,columns=Y_test.columns)

In [79]:
for col in Y_test.columns:
    print(col)
    print(classification_report(Y_test[col], y_pred_cv[col]))

related
              precision    recall  f1-score   support

         0.0       0.40      0.04      0.07      1503
         1.0       0.77      0.98      0.86      5000
         2.0       0.00      0.00      0.00        49

    accuracy                           0.76      6552
   macro avg       0.39      0.34      0.31      6552
weighted avg       0.68      0.76      0.67      6552

request
              precision    recall  f1-score   support

         0.0       0.83      0.99      0.90      5395
         1.0       0.37      0.03      0.05      1157

    accuracy                           0.82      6552
   macro avg       0.60      0.51      0.48      6552
weighted avg       0.75      0.82      0.75      6552

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6530
         1.0       0.00      0.00      0.00        22

    accuracy                           1.00      6552
   macro avg       0.50      0.50      0.50      655

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      6330
         1.0       0.00      0.00      0.00       222

    accuracy                           0.97      6552
   macro avg       0.48      0.50      0.49      6552
weighted avg       0.93      0.97      0.95      6552

child_alone
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6552

    accuracy                           1.00      6552
   macro avg       1.00      1.00      1.00      6552
weighted avg       1.00      1.00      1.00      6552

water
              precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      6119
         1.0       0.05      0.00      0.00       433

    accuracy                           0.93      6552
   macro avg       0.49      0.50      0.48      6552
weighted avg       0.88      0.93      0.90      6552

food
              precision    recall  f1-score   supp

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.95      1.00      0.98      6255
         1.0       0.29      0.01      0.01       297

    accuracy                           0.95      6552
   macro avg       0.62      0.50      0.49      6552
weighted avg       0.92      0.95      0.93      6552

other_aid
              precision    recall  f1-score   support

         0.0       0.87      0.99      0.93      5710
         1.0       0.08      0.01      0.01       842

    accuracy                           0.86      6552
   macro avg       0.47      0.50      0.47      6552
weighted avg       0.77      0.86      0.81      6552

infrastructure_related
              precision    recall  f1-score   support

         0.0       0.94      1.00      0.97      6127
         1.0       0.14      0.00      0.00       425

    accuracy                           0.93      6552
   macro avg       0.54      0.50      0.49      6552
weighted avg       0.88      0.93      0.

...

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjha

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6477
         1.0       0.00      0.00      0.00        75

    accuracy                           0.99      6552
   macro avg       0.49      0.50      0.50      6552
weighted avg       0.98      0.99      0.98      6552

other_infrastructure
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      6270
         1.0       0.17      0.00      0.01       282

    accuracy                           0.96      6552
   macro avg       0.56      0.50      0.49      6552
weighted avg       0.92      0.96      0.94      6552

weather_related
              precision    recall  f1-score   support

         0.0       0.74      0.96      0.84      4732
         1.0       0.58      0.13      0.22      1820

    accuracy                           0.73      6552
   macro avg       0.66      0.55      0.53      6552
weighted avg       0.70      0.73    

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjha

              precision    recall  f1-score   support

         0.0       0.92      1.00      0.95      5928
         1.0       0.74      0.13      0.22       624

    accuracy                           0.91      6552
   macro avg       0.83      0.56      0.59      6552
weighted avg       0.90      0.91      0.88      6552

cold
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      6421
         1.0       0.00      0.00      0.00       131

    accuracy                           0.98      6552
   macro avg       0.49      0.50      0.49      6552
weighted avg       0.96      0.98      0.97      6552

other_weather
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97      6197
         1.0       0.00      0.00      0.00       355

    accuracy                           0.94      6552
   macro avg       0.47      0.50      0.49      6552
weighted avg       0.89      0.94      0.92      6552



C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [80]:
cv.score(X_test, Y_test)

0.15827228327228327

In [81]:
np.mean(np.mean(Y_test.reset_index(drop=True).eq(y_pred_cv), axis=0))

0.9249592999592999

In [101]:
df_add = Y_test.reset_index(drop=True).add(y_pred_cv, fill_value=0)
(df_add[df_add == 2]/2).sum(axis=0, numeric_only=True).sum()/(Y_test.sum(axis=0, numeric_only=True).sum())

0.27828075860097207

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [55]:
pipeline2 = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=KNeighborsClassifier()))
])

In [56]:
pipeline2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001F7A8F769D0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=KNeighborsClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001F7A8F769D0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=KNeighborsClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tf

In [57]:
'''
First Run of GridSearchCV with KNearestNeighbors
1/18/21 3PM
parameters = {'clf__estimator__n_neighbors': [5,7],
              'clf__estimator__weights': ['uniform', 'distance']}

The best parameters across ALL searched params:
 {'clf__estimator__n_neighbors': 7, 'clf__estimator__weights': 'uniform'}              
'''

parameters = {'clf__estimator__n_neighbors': [7, 9, 11]}

cv2 = GridSearchCV(estimator=pipeline2, param_grid=parameters, n_jobs=6, cv=3, verbose=5)

In [58]:
cv2.fit(X_train, Y_train)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001F7A8F769D0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=KNeighborsClassifier()))]),
             n_jobs=6, param_grid={'clf__estimator__n_neighbors': [7, 9, 11]},
             verbose=5)

In [59]:
print(" Results from Grid Search (model name: cv2)" )
print("\n The best estimator across ALL searched params:\n",cv2.best_estimator_)
print("\n The best score across ALL searched params:\n",cv2.best_score_)
print("\n The best parameters across ALL searched params:\n",cv2.best_params_)

 Results from Grid Search (model name: cv2)

 The best estimator across ALL searched params:
 Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001F7A8F769D0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(n_neighbors=11)))])

 The best score across ALL searched params:
 0.1638262996070964

 The best parameters across ALL searched params:
 {'clf__estimator__n_neighbors': 11}


In [60]:
y_pred2 = cv2.predict(X_test)

In [61]:
y_pred2 = pd.DataFrame(y_pred2,columns=Y_test.columns)

In [62]:
for col in Y_test.columns:
    print(col)
    print(classification_report(Y_test[col], y_pred2[col]))

related
              precision    recall  f1-score   support

         0.0       0.29      0.12      0.17      1503
         1.0       0.77      0.92      0.84      5000
         2.0       0.33      0.04      0.07        49

    accuracy                           0.73      6552
   macro avg       0.47      0.36      0.36      6552
weighted avg       0.66      0.73      0.68      6552

request
              precision    recall  f1-score   support

         0.0       0.83      0.98      0.90      5395
         1.0       0.37      0.05      0.09      1157

    accuracy                           0.82      6552
   macro avg       0.60      0.52      0.49      6552
weighted avg       0.75      0.82      0.76      6552

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6530
         1.0       0.00      0.00      0.00        22

    accuracy                           1.00      6552
   macro avg       0.50      0.50      0.50      655

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjha

              precision    recall  f1-score   support

         0.0       0.59      0.89      0.71      3815
         1.0       0.45      0.13      0.20      2737

    accuracy                           0.57      6552
   macro avg       0.52      0.51      0.45      6552
weighted avg       0.53      0.57      0.49      6552

medical_help
              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      6032
         1.0       0.00      0.00      0.00       520

    accuracy                           0.92      6552
   macro avg       0.46      0.50      0.48      6552
weighted avg       0.85      0.92      0.88      6552

medical_products
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97      6217
         1.0       0.00      0.00      0.00       335

    accuracy                           0.95      6552
   macro avg       0.47      0.50      0.49      6552
weighted avg       0.90      0.95      0.92 

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjha

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      6428
         1.0       0.00      0.00      0.00       124

    accuracy                           0.98      6552
   macro avg       0.49      0.50      0.50      6552
weighted avg       0.96      0.98      0.97      6552

military
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      6330
         1.0       0.00      0.00      0.00       222

    accuracy                           0.97      6552
   macro avg       0.48      0.50      0.49      6552
weighted avg       0.93      0.97      0.95      6552

child_alone
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6552

    accuracy                           1.00      6552
   macro avg       1.00      1.00      1.00      6552
weighted avg       1.00      1.00      1.00      6552

water
              precision    recall  f1-score   

...

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjha

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      6477
         1.0       0.00      0.00      0.00        75

    accuracy                           0.99      6552
   macro avg       0.49      0.50      0.50      6552
weighted avg       0.98      0.99      0.98      6552

other_infrastructure
              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      6270
         1.0       0.00      0.00      0.00       282

    accuracy                           0.96      6552
   macro avg       0.48      0.50      0.49      6552
weighted avg       0.92      0.96      0.94      6552

weather_related
              precision    recall  f1-score   support

         0.0       0.75      0.92      0.83      4732
         1.0       0.50      0.20      0.29      1820

    accuracy                           0.72      6552
   macro avg       0.63      0.56      0.56      6552
weighted avg       0.68      0.72    

C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjha

              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97      6197
         1.0       0.00      0.00      0.00       355

    accuracy                           0.95      6552
   macro avg       0.47      0.50      0.49      6552
weighted avg       0.89      0.95      0.92      6552

direct_report
              precision    recall  f1-score   support

         0.0       0.81      0.98      0.89      5279
         1.0       0.39      0.05      0.08      1273

    accuracy                           0.80      6552
   macro avg       0.60      0.51      0.49      6552
weighted avg       0.73      0.80      0.73      6552



C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\sjhal\Anaconda3\envs\disaster\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [63]:
cv2.score(X_test, Y_test)

0.1704822954822955

In [71]:
np.mean(np.mean(Y_test.reset_index(drop=True).eq(y_pred2), axis=0))

0.9245904558404558

In [100]:
df_add = Y_test.reset_index(drop=True).add(y_pred2, fill_value=0)
(df_add[df_add == 2]/2).sum(axis=0, numeric_only=True).sum()/(Y_test.sum(axis=0, numeric_only=True).sum())

0.26265129133708187

### 9. Export your model as a pickle file

In [103]:
# save the model to disk
filename = 'classifier.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.